In [3]:
import os
import json
import requests
from openai import OpenAI
import time
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv())
fmp_key = os.environ['FMP_API_KEY']
# print(fmp_key)

client : OpenAI = OpenAI()

In [4]:
# Define financial statement functions
def get_income_statement(ticker, period, limit):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())


# Map available functions
available_functions = {
    "get_income_statement": get_income_statement,
    # "get_balance_sheet": get_balance_sheet,
    # "get_cash_flow_statement": get_cash_flow_statement,
    # "get_key_metrics": get_key_metrics,
    # "get_financial_ratios": get_cash_flow_statement,
    # "get_financial_growth": get_financial_ratios
}

In [14]:
import inspect

# Define the main function
def run_assistant(user_message):
    assistant = client.beta.assistants.create(
        name="Financial Bot",
        instructions="Act as a financial analyst by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends. ",
        model="gpt-3.5-turbo-1106",
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "get_income_statement",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"},
                        },
                    },
                },
            },
            # same for the rest of the financial functions
        ],
    )
      # Creating a new thread
    thread = client.beta.threads.create()

    # Adding a user message to the thread

    user_message = "I need to solve the equation `3x + 11 = 14`. Can you help me?"

    client.beta.threads.messages.create(
          thread_id=thread.id,
          role="user",
          content=user_message
    )

    # Running the assistant on the created thread
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

        # Add run steps retrieval here
        run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
        print("Run Steps:", run_steps)

        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                if function_name in available_functions:
                    function_to_call = available_functions[function_name]

                    # Check if 'limit' is a required parameter for the function
                    required_parameters = inspect.signature(function_to_call).parameters
                    if 'limit' in required_parameters:
                        # Add 'limit' to function_args if it's required
                        function_args['limit'] = 10  # Replace 10 with the desired limit value

                    output = function_to_call(**function_args)
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output,
                    })

            # Check if tool_outputs is not empty before submitting
            if tool_outputs:
                # Submit tool outputs and update the run
                client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread.id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )
            else:
                print("No tool outputs to submit.")

        elif run.status == "completed":
            # List the messages to get the response
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            for message in messages.data:
                role_label = "User" if message.role == "user" else "Assistant"
                message_content = message.content[0].text.value
                print(f"{role_label}: {message_content}\n")
            break  # Exit the loop after processing the completed run

        elif run.status == "failed":
            print("Run failed.")
            break

        elif run.status in ["in_progress", "queued"]:
            print(f"Run is {run.status}. Waiting...")
            time.sleep(5)  # Wait for 5 seconds before checking again

        else:
            print(f"Unexpected status: {run.status}")
            break
        
run_assistant('What is Financial?')

Run Steps: SyncCursorPage[RunStep](data=[], object='list', first_id=None, last_id=None, has_more=False)
Run is in_progress. Waiting...
Run Steps: SyncCursorPage[RunStep](data=[RunStep(id='step_QoclfQ0tmrUZqyV0VpFg8yol', assistant_id='asst_kDJJsoavRQvIfEqFX5ezBDsy', cancelled_at=None, completed_at=None, created_at=1703242802, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_KDjKTHEyL3I7JIJTJAYU5RIx', status='in_progress', step_details=MessageCreationStepDetails(message_creation=None, type='tool_calls', tool_calls=[{'id': 'call_PXrAQzrqASHwPF63gow7wLST', 'type': 'function', 'function': {'name': 'solve_equation', 'arguments': '{"equation": "3x + 11 = 14"}'}}, {'id': 'call_Uvmd6P2i2pcalGETe0WMHvNQ', 'type': 'function', 'function': {'name': 'solve_equation', 'arguments': '{"equation": "3x + 11 = 14"}'}}]), thread_id='thread_rjBeWGQYkOngB1RLgpxEJ6TQ', type='tool_calls', expires_at=1703243401)], object='list', first_id='step_QoclfQ0tmrUZqy